# 75.06 Organización de datos: Trabajo Práctico 1
Integrantes del grupo
- Avecilla, Ignacio - 105067
- Balmaceda, Fernando - 105525
- Singer, Joaquín - 105854
- Villegas, Tomás - 106456

# Parte 1: Entrada en calor

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
import os

ds_path = None

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if "recent-grads.csv" in filename:
            ds_path = dirname+"/"+filename

## Análisis exploratorio

### Exploración inicial

In [ ]:
#Creamos el data frame
df = pd.read_csv(ds_path)

#Mostramos nombres y tipos de las columnas
df.info()

#### Descripcion para cada columna:
- Rank -> Numero identificador para cada especializacion.
- Major_code -> Codigo identificador para la especializacion.
- Major_category -> Categoria a la que pertenece la especializacion.
- Total -> Total de alumnos que participan de la especializacion.
- Sample_size -> Cantidad de instituciones de las que se obtuvieron los datos para la especializacion.
- Men -> Total de varones que participan de la especializacion.
- Women -> Total de mujeres que participan de la especializacion.
- ShareWomen -> Porcentaje de mujeres entre el total de alumnos.
- Employed -> Cantidad de personas que tienen trabajo que participan de esa especializacion.
- Full_time -> Cantidad de personas con un trabajo full time que participan de esa especializacion.
- Part_time -> Cantidad de personas con un trabajo part time que participan de esa especializacion.
- Full_time_year_round -> Cantidad de personas con un trabajo Full time year round que participan de esa especializacion.
- Unemployed -> Cantidad de personas que no tienen trabajo que participan de esa especializacion.
- Unemplyment_rate -> Ratio de desempleo entre los participantes de esa especializacion.
- Median -> Mediana del salario del total de las personas que participan de la especializacion.
- P25th -> Primer quartil del salario del total de las personas que participan de la especializacion.
- P75th -> Segundo quartil del salario del total de las personas que participan de la especializacion.
- College_jobs -> Cantidad de personas trabajando en la institucion.
- Non_college_jobs -> Cantidad de personas trabajajando fuera de la institucion.
- Low_wage_jobs -> Cantidad de personas trabajando en lugares donde el salario es bajo.

In [ ]:
#Mostramos las primeras filas
df.head()

In [ ]:
#Mostramos las últimas filas
df.tail()

In [ ]:
#Mostramos la cantidad de valores nulos por columna
df.isna().sum()

Como podemos ver no hay valores nulos en ninguna columna salvo en la columna "Unemployment_rate" donde existe un valor nulo.

### Variables cualitativas

In [ ]:
# Armamos el nuevo dataframe para major_category
df1 = pd.DataFrame(df["Major_category"].value_counts()).reset_index()
columns_change = {"index":"Category", "Major_category":"Frequency"}
df1.rename(columns=columns_change, inplace=True)
df1

Graficamos los valores existentes con un grafico de barras

In [ ]:
plt.figure(figsize = (15,8))
sns.set(font_scale = 1.5)
ax = sns.countplot(y="Major_category", data=df, color="skyblue", order = df['Major_category'].value_counts().index, )
ax.set_xlabel("Frequency", fontsize = 20)
_ = ax.set_ylabel("Major Category", fontsize = 20)

#### ¿Por qué no podemos hacer un histograma con "Major_category"? ¿En que se diferencian estos con los gráficos de barra?
Porque los histogramas se realizan con variables continuas, como un precio por ejemplo. Category es una variable discreta.

En un histograma, se toman rangos continuos agrupados en barras llamadas bins. En cambio, en un gráfico de barras se toman rangos discretos. 

### Variables cuantitativas

In [ ]:
#Creamos un nuevo data set con los valores pedidos
df1 = pd.DataFrame(index=["First Quantile", "Third Quantile", "Mean", "Median", "Mode", "Range"])
for i in df:
    if df[i].dtype=="int64" or df[i].dtype=="float64":
        data = [
            round(df[i].quantile(0.25), 2),
            round(df[i].quantile(0.75), 2),
            round(df[i].mean(), 2),
            round(df[i].median(), 2),
            round(df[i].mode()[0], 2),
            round(df[i].max() - df[i].min(), 2)
        ]
        df1[i]=data
df1

In [ ]:
#scatter matrix de algunas de las variables cuantitativas
df_aux = df.copy()

columns = ["Total", "Employed", "Full_time","Part_time", "Unemployed", "College_jobs", "Low_wage_jobs"]
df_for_scatter = df_aux[columns].copy()

scatter_matrix(df_for_scatter, figsize=(20,20))
plt.show()

In [ ]:
# Calculamos la correlacion de Pearson para cada columna
pearson_correlation = df.corr(method="pearson")
pearson_correlation

In [ ]:
#heatmap con la correlación de pearson
plt.figure(figsize = (10,8))
sns.set(font_scale = 1)
ax = sns.heatmap(pearson_correlation)

## Major categories

- Engineering
- Business
- Physical Sciences
- Law & Public Policy
- Computers & Mathematics
- Agriculture & Natural Resources
- Industrial Arts & Consumer Services
- Arts
- Health
- Social Science
- Biology & Life Science
- Education
- Humanities & Liberal Arts
- Psychology & Social Work
- Communications & Journalism
- Interdisciplinary

In [ ]:
categories_array = []
def categories_to_array(category):
    categories_array.append(category) if category not in categories_array else None
df["Major_category"].map(categories_to_array)

df_aux1 = pd.DataFrame(columns = df.columns)
df_aux2 = pd.DataFrame(columns = df.columns)

for i in df.index:
    if df["Major_category"][i] in categories_array[0:8]:
        df_aux1 = df_aux1.append(df.loc[i,:])
    else:
        df_aux2 = df_aux2.append(df.loc[i,:])

plt.figure(figsize = (40,15))
ax = sns.boxplot(x="Major_category",y = (df["Employed"] / df["Total"]) * 100, data=df_aux1)
plt.xlabel("Major Category")
plt.ylabel("Employment rate (%)")
plt.ylim([60,100])
plt.title("Porcentaje de personas empleadas por major category")
plt.show()

plt.figure(figsize = (40,15))
ax = sns.boxplot(x="Major_category",y = (df["Employed"] / df["Total"]) * 100, data=df_aux2)
plt.xlabel("Major Category")
plt.ylabel("Employment rate (%)")
plt.ylim([60,100])
plt.title("Porcentaje de personas empleadas por major category")
plt.show()

Observando ambos graficos se puede concluir:
1. Las categorias **Engineering**, **Phisical Sciences**, **Health** y **Biology and Life Science** son las que mas densidad de empleo tienen.
2. Las categorias **Agriculture & Natural Resources** y **Education** son las que concentran mayor porcentaje de empleo, siendo las 2 más cercanas al 90%.

In [ ]:
fig, axes = plt.subplots()
axes.violinplot(dataset = pd.DataFrame(
    data={
        "Full_time_rate":(df["Full_time"]/df["Total"])*100,
        "Part_time_rate":(df["Part_time"]/df["Total"])*100
    }
))
axes.yaxis.grid(True)
axes.set_xticklabels([])
axes.set_xlabel('Full time VS Part time')
axes.set_ylabel('Porcentaje')

plt.show()

Se puede observar que hay mayor densidad de empleados **Full time** a comparacion de **Part time**.

In [ ]:
plt.figure(figsize = (12,8))
sns.set(font_scale = 1.3)
ax = sns.histplot(data=(df["Employed"]/df["Total"])*100, palette = "rocket")
ax.set_xticks(np.arange(0, 101, 5))
plt.title("Frecuencia de porcentaje de empleados en general")
plt.xlabel("Porcentaje de empleados")
plt.ylabel("Frecuencia")
plt.show()

Se puede observar que en general tiende a haber un alto porcentaje de empleo, concentrandose entre el 75% y 90%.

In [ ]:
plt.figure(figsize = (10,6))
sns_hist=sns.kdeplot(data=df['Unemployment_rate'] * 100).set(title="Densidad de desempleo por cada major",xlabel="% de desempleo")
plt.ylabel("")
_ = plt.yticks([])

Podemos observar que por cada Major el porcentaje de desempleo se encuentra entre un 5% y un 10% en su mayoria

In [ ]:
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

fig = px.scatter(
    df, x="College_jobs",
    y="Low_wage_jobs",
    size="Employed",
    color="Median",
    hover_name="Major",
    log_x=True,
    size_max=60
)
fig.show()

Con este grafico podemos determinar que a medida que aumentan los college jobs hay ciertos majors que aumentan sus trabajos mal pagos pero por otro lado hay muchos majors donde estas dos variables no estan necesariamente correlacionadas

# Conclusiones

Como fuimos analizando a lo largo de cada grafico, obtenemos mas informacion sobre la tasa de empleo y desempleo ademas de poder discriminar como se comporta en cada major y major category.
Gracias al scatter matrix y al heatmap podemos ver que existe mucha correlacion en las variables que se relacionan al trabajo (Employed, Full_time, Part_time, Unemployed, Full_time_year_round) todas estas variables estan bastante correlacionadas entre si asi que aumentan ambas de manera muy similar. Todas estas variables tambien se relacionan con "Total" que tiene todo el sentido ya que si aumenta la cantidad de alumnos de un major hay mayor cantidad de gente que este empleada y desempleada ademas de tener un tipo de contrato de trabajo.

# Parte 2: Preguntas de investigación

## Exploración del dataset

In [ ]:
!pip install openpyxl

In [ ]:
#Creamos el data frame y mostramos sus primeras y últimas filas
df_spotify = pd.read_excel("https://drive.google.com/uc?export=download&id=1PiUr7g_FZn3IL9tQ7G-8Ylzl2LYrnZoW")
df_spotify

In [ ]:
#Mostramos nombres y tipos de las columnas
df_spotify.info()

In [ ]:
#Mostramos la cantidad de valores nulos por columna
df_spotify.isna().sum()

Observamos que no hay ningun valor nan por lo tanto podemos trabajar con comodidad en el analisis de datos

In [ ]:
plt.figure(figsize = (10,6))
ax = sns.countplot(y="genre", data=df_spotify, color="skyblue", order = df_spotify['genre'].value_counts().index)
plt.xlabel("Count")
plt.ylabel("Genre")
plt.show()

Podemos observar que la cantidad de ocurrencias que tenemos de cada genero es bastante pareja

In [ ]:
#scatter matrix de todas las variables cuantitativas
df_aux = df_spotify.copy()

columns = df_aux.columns.tolist()
columns.remove("genre")
df_for_scatter = df_aux[columns].copy()

scatter_matrix(df_for_scatter, figsize=(30,28));

Hacemos este scatter matrix para poder ir familiarizandonos con como se relacionan nuestras variables

## Planteo de las preguntas

A partir de nuestra exploración inicial, nos pareció interesante trabajar con las siguientes dos preguntas:

¿Como se relaciona la duracion de las canciones con las demas variables y como afecta a la popularidad?
¿Cual es el genero mas popular y por que lo es?

## Investigación y gráficos

### ¿Como se relaciona la duracion de las canciones con las demas variables y como afecta a la popularidad?

Cuando comenzamos a planear el abordaje de la pregunta, nos dimos cuenta de que trabajar directamente con la duración en segundos de las canciones no nos iba a dar la información que queríamos, por lo que decidimos trabajar con rangos discretos.

In [ ]:
duraciones = pd.DataFrame(df_aux["duration_ms"]).describe()

separacion_duraciones = {
    "25%":duraciones["duration_ms"]["25%"],
    "50%":duraciones["duration_ms"]["50%"], 
    "75%":duraciones["duration_ms"]["75%"]  
}
# 0 - 25 => muy corto
# 25 - 50 => corto
# 50 - 75 => normal
# 75 o más => largo 

def duration_parser(duration: float):
    if duration < separacion_duraciones["25%"]:
        return "very short"
    elif duration >= separacion_duraciones["25%"] and duration < separacion_duraciones["50%"]:
        return "short"
    elif duration >= separacion_duraciones["50%"] and duration < separacion_duraciones["75%"]:
        return "normal"
    else:
        return "large"

In [ ]:
column = df_aux["duration_ms"].apply(duration_parser)
df_aux["duration_category"] = column
df_aux

Con la divisón ya realizada, primero vamos a visualizar la situación con un boxplot que relacione estas categorías con la popularidad

In [ ]:
ax = sns.boxplot(x="duration_category",y = "popularity", data=df_aux)
plt.xlabel("Duration category")
plt.ylabel("Popularity")
plt.title("Popularidad segun duracion")
plt.show()

Podemos ver que la popularidad no varia mucho segun la duracion de la cancion por lo menos en los rangos que definimos. Quizas las canciones mas largas presentan una menor probabilidad de tener una alta popularidad comparada con los demas valores y las cortas son las que mas populares llegan a ser salvo por algun que otro valor atipico en las otras categorias

In [ ]:
# Hacemos un bar plot para ver cuantas canciones de cada duracion hay segun el genero
plt.figure(figsize = (20,10))
sns.set(font_scale = 1.3)
ax = sns.countplot(x="genre",hue="duration_category", data=df_aux, palette="rocket")
plt.title("Cantidad de canciones de cierta duracion por genero")
plt.legend(title="Duration Category")
plt.show()

Con esto podemos determinar la cantidad de canciones con cierta duracion presente en cada genero. Podriamos decir que el genero World y el Classical van a tener probabilidad de ser menos popular ya que la mayoria de sus canciones son largas. En cambio generos como el reggaeton y el Pop tienen gran presencia de canciones de corta y muy corta duracion por lo tanto tienen probabilidad de ser populares.

In [ ]:
ax = sns.violinplot(x="duration_category",y = "energy", data=df_aux)
plt.xlabel("Duration category")
plt.ylabel("Energy")
plt.title("Energia segun duracion")
plt.show()

Analizando la energia segun la duracion de cada cancion observamos que las canciones cortas tienden a tener mas energia y va disminuyendo a medida que la cancion es mas larga

In [ ]:
ax = sns.boxplot(x="duration_category",y = "danceability", data=df_aux)
plt.xlabel("Duration category")
plt.ylabel("Danceability")
plt.title("Danzabilidad segun duracion")
plt.show()

Como podemos observar las canciones mas cortas son las que mas predominan en altas danzabilidades y disminuye nuevamente a medida que aumenta la duracion. Podemos mencionar que en esta comparacion es donde observamos mas valores atipicos. En duraciones cortas hay muchas canciones que tienden a tener muy poca danzabilidad.

In [ ]:
ax = sns.stripplot(y="duration_category", x="tempo", data=df_aux, jitter=0.05)
plt.ylabel("Duration category")
plt.xlabel("Tempo")
plt.title("Tempo segun duracion")
plt.show()

Podemos ver que el tempo es bastante parecido en casi todos los rangos de duracion, no hay grandes cambios aunque si podemos notar que las canciones largas no estan presentes en tempos muy altos

In [ ]:
ax = sns.violinplot(y="duration_category",x = "loudness", data=df_aux)
plt.ylabel("Duration category")
plt.xlabel("Loudness")
plt.title("Loudness segun duracion")
plt.show()

Podemos ver que las canciones que tienen mayor loudness son las que se encuentran en los extremos de las categorias de duracion, es decir que las mas cortas y las mas largas son las que tienen mayor loudness mientas que las cortas y normales son las que se mantienen en un loudness mas bajo.

In [ ]:
ax = sns.boxplot(x="duration_category",y = "acousticness", data=df_aux)
plt.ylabel("Duration category")
plt.xlabel("Acousticness")
plt.title("Acousticness segun duracion")
plt.show()

Nuevamente podemos observar que las canciones con mayor acousticness se encuentran en los extremos, en las canciones mas cortas y mas largas mientras que los valores de duracion del medio se encuentran en rangos mas bajos.

## Conclusiones

Podemos observar cierta tendencia a que las canciones que se mueven entre rangos de duracion cortos y normales suelen tener un comportamiento parecido mientras que los extremos muy cortos o largos se mueven en los mismos valores en casi todas las relaciones con las demas variables.

# ¿Cual es el genero mas popular y por que lo es?

Ahora vamos a buscar las caracteristicas de un genero para poder ser popular, ademas vamos a poder relacionar con los resultados anteriores a ver si las canciones de cierta duracion buscan ser mas populares o simplemente las caracteristicas de esas canciones son asi por coincidencia.

In [ ]:
#scatter matrix de la popularidad con las demas variables
df_aux = df_spotify.copy()
ax = sns.pairplot(
    df_aux,
    y_vars=["popularity"],
)

Si bien ya hicimos un scattermatrix al principio, ahora nos interesa focalizarnos unicamente en las relaciones con la popularidad asi que vamos a ver esas relaciones especificamente

In [ ]:
plt.figure(figsize = (20,8))
my_order = df_aux.groupby("genre")["popularity"].median().sort_values().iloc[::-1].index
ax = sns.boxplot(x="genre",y = "popularity", data=df_aux, order= my_order)
plt.xlabel("Genre")
plt.ylabel("Popularity")
plt.title("Popularidad segun Genero")
plt.show()

Gracias a esto podemos ver que genero es el que tiene promedio de mayor popularidad comparado con los otros. Vemos que el pop es sin duda el de mayor probabilidad de popularidad junto con Dance, Hip-Hop y rock. Vemos que el reggaeton es el que tiene un mayor rango entre sus canciones populares y las que no lo son y la opera es el menos popular.

### Analisis de los generos mas populares

In [ ]:
plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Pop"]
sns.histplot(data=df_pop, x="energy")
plt.title("Energia de canciones pop")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Rock"]
sns.histplot(data=df_pop, x="energy")
plt.title("Energia de canciones Rock")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Dance"]
sns.histplot(data=df_pop, x="energy")
plt.title("Energia de canciones Dance")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Hip-Hop"]
sns.histplot(data=df_pop, x="energy")
plt.title("Energia de canciones Hip-Hop")
plt.show()

Como vemos en el scattermatrix del principio, la energia parece estar en cierta medida correlacionada con la popularidad y al ver la energia de las canciones de estos generos vemos que se mueven en su mayoria en valores medianos/altos asi que esta puede ser una variable a tener en cuenta a la hora de hacer una cancion popular.

In [ ]:
plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Pop"]
sns.histplot(data=df_pop, x="danceability")
plt.title("Energia de canciones pop")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Rock"]
sns.histplot(data=df_pop, x="danceability")
plt.title("Energia de canciones Rock")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Dance"]
sns.histplot(data=df_pop, x="danceability")
plt.title("Energia de canciones Dance")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Hip-Hop"]
sns.histplot(data=df_pop, x="danceability")
plt.title("Energia de canciones Hip-Hop")
plt.show()

Algo muy similar pasa con la danzabilidad en las canciones de estos generos y la popularidad se relaciona en cierta medida con estas caracteristicas. En este caso el genero del pop y el rock mantiene una danzabilidad predominante en los valores medios, Dance obtiene el promedio mas alto y Rock se mantiene dentro de todo parejo salvo en un valor particular.

In [ ]:
plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Pop"]
sns.histplot(data=df_pop, x="loudness")
plt.title("Energia de canciones pop")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Rock"]
sns.histplot(data=df_pop, x="loudness")
plt.title("Energia de canciones Rock")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Dance"]
sns.histplot(data=df_pop, x="loudness")
plt.title("Energia de canciones Dance")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Hip-Hop"]
sns.histplot(data=df_pop, x="loudness")
plt.title("Energia de canciones Hip-Hop")
plt.show()

La caracteristica de loudness se correlaciona con la popularidad, de tal manera que cuanto mas cercano es el loudness a 0 mas probabilidad de ser popular. A la hora de analizar el loudness de los generos vemos que se mantiene en los valores mas cercanos a 0 comparado con los valores posibles, salvo por el Hip-Hop que es el genero que tiene la mayoria de los valores del loudness cercanos a -10 por lo cual puede no ayudar a la popularidad en las canciones de este genero

In [ ]:
plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Pop"]
sns.histplot(data=df_pop, x="speechiness")
plt.title("Energia de canciones pop")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Rock"]
sns.histplot(data=df_pop, x="speechiness")
plt.title("Energia de canciones Rock")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Dance"]
sns.histplot(data=df_pop, x="speechiness")
plt.title("Energia de canciones Dance")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Hip-Hop"]
sns.histplot(data=df_pop, x="speechiness")
plt.title("Energia de canciones Hip-Hop")
plt.show()

La relacion de speechiness y popularidad muestra que en los valores mas bajos cercanos a 0 no hay mucha correlacion pero a medida que sube el speechiness no obtenemos valores de popularidad bajos. En este caso para las canciones de estos generos los valores son todos cercanos a 0 en su mayoria asi que esta caracteristica no nos da mucha informacion sobre la popularidad de las canciones de este genero. El unico que crece un poco de speechiness es el Hip-Hop 

In [ ]:
plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Pop"]
sns.histplot(data=df_pop, x="acousticness")
plt.title("Energia de canciones pop")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Rock"]
sns.histplot(data=df_pop, x="acousticness")
plt.title("Energia de canciones Rock")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Dance"]
sns.histplot(data=df_pop, x="acousticness")
plt.title("Energia de canciones Dance")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Hip-Hop"]
sns.histplot(data=df_pop, x="acousticness")
plt.title("Energia de canciones Hip-Hop")
plt.show()

Como vemos en el scatter matrix, el acousticness de una cancion puede considerarse que se correlaciona negativamente con la popularidad. Como vemos en estos graficos, los generos mas populares tienen un acousticness muy cercano a 0 por lo tanto tienen mas probabilidad de obtener una cancion popular.

### Analisis del genero menos popular

A continuacion vamos a investigar las mismas caracteristicas pero con el genero menos popular, para ver si tiene sentido decir que estas variables influyen efectivamente en la popularidad de un genero

In [ ]:
plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Opera"]
sns.histplot(data=df_pop, x="energy")
plt.title("Energia de canciones Opera")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Opera"]
sns.histplot(data=df_pop, x="danceability")
plt.title("Energia de canciones Opera")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Opera"]
sns.histplot(data=df_pop, x="loudness")
plt.title("Energia de canciones Opera")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Opera"]
sns.histplot(data=df_pop, x="speechiness")
plt.title("Energia de canciones Opera")
plt.show()

plt.figure(figsize = (8,5))
df_pop = df_aux[df_aux["genre"] == "Opera"]
sns.histplot(data=df_pop, x="acousticness")
plt.title("Energia de canciones Opera")
plt.show()

Como vemos la opera difiere en absolutamente todas las caracteristicas comparadas con los generos mas populares y no cumple con los valores que tienen relacion con una mayor popularidad

# Conclusiones

Gracias a estos analisis podemos concluir en que los generos mas populares comparten valores muy parecidos en las mismas caracteristicas y el genero menos popular contrasta completamente con dichos valores. Asi que la popularidad de un genero o una cancion va a deberse en gran medida por tener:
- Una energia de 0.6 para adelante.
- Una bailabilidad de 0.5 en adelante aproximadamente.
- Un volumen mayor a -8 aproximadamente.
- Una acustica cercana lo mas posible a 0.

Como vemos en los ultimos graficos, la opera presenta valores totalmente contrarios a estas caracteristicas que hayamos:
- Una energia predominantemente de 0.
- Una bailabilidad muy cercana a 0.
- Un volumen predominante en -20.
- Una acustica muy cercana a 1.